In [1]:
import os
import requests
import json
import pandas as pd


/home/tesa/thegraph/subgraphs


### TheGraph Explorer

To pull the query from a subgraph, you'll want to use the explorer to generate a query that you want to use and copy the api under "Queries(HTTP)".

<img src="./images/thegraph_uniswapv2.png">

On the right hand of the page, you can click on the entity names and see the items available to query.  The query is written in **schema.graphql**. 

After you update the query with the fields that you are interested in, run the explorer using the purple button.

<img src="./images/uniswapv2_runquery.png">

### Explorer -> Python

Copy and paste the entire query into Python.  Since the format of **schema.graphql** needs to be preserved, add backslashes to the end of each line.

** Note that the query is a plural "uniswapFactories" while the entity is a singular "UniswapFactory".  This goes the same for any entity ie. "mints" query -> "mint" entity, "burns" query -> "burn" entity.

In [15]:
## copy and past the url and query from the explorer
subgraph_url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"
query = "{\
  uniswapFactories(first: 5) {\
    id\
    pairCount\
    totalVolumeUSD\
    mostLiquidTokens{\
        id\
    }\
  }\
}"

### GraphQuery class

The get_query function is a POST request that returns the data in a json format.  

** Note that some numeric fields are returned as strings and not as floats or ints.  

In [13]:
class GraphQuery(object):
    def __init__(self):
        self.data = None

    def get_query(self, subgraph_url, query, variables=None):
        """
        :param subgraph_url: 'https://api.thegraph.com/subgraphs/name/ppunky/hegic-v888'
        :param query: '{options(where: {status:"ACTIVE"}) {id symbol}}'
        :param variables:
        :return:
        """
        request_json = {'query': query}
        if variables:
            request_json['variables'] = variables
        resp = requests.post(subgraph_url, json=request_json)
        data = json.loads(resp.text)
        self.data = data['data']
        return self.data

In [16]:
### running the query
gq = GraphQuery()
data = gq.get_query(subgraph_url, query)
data

{'uniswapFactories': [{'id': '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f',
   'mostLiquidTokens': [],
   'pairCount': 23147,
   'totalVolumeUSD': '35417018263.02062294114348020249348'}]}

In [17]:
### convert the data into a pandas dataframe
df = pd.DataFrame(data.get('uniswapFactories', []))
df

,id,mostLiquidTokens,pairCount,totalVolumeUSD
0,0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f,[],23147,35417018263.02062294114348020249348


In [19]:
### check the data type
df.dtypes

id                  object
mostLiquidTokens    object
pairCount            int64
totalVolumeUSD      object
dtype: object

In [20]:
### change the data type
df["totalVolumeUSD"] = df.totalVolumeUSD.astype(float)
df.dtypes

id                   object
mostLiquidTokens     object
pairCount             int64
totalVolumeUSD      float64
dtype: object